In [9]:
from collections.abc import MutableMapping
import math
import random


class SkipList(MutableMapping):
    __slots__ = '_head', '_tail', '_n', '_height'

    #------------------------------- nested _Node class -------------------------------
    class _Node:
        __slots__ = '_key', '_value', '_next'

        """Lightweight composite to store key-value pairs as map items."""

        def __init__(self, k, v, height):
            self._key = k
            self._value = v
            self._next = [None] * (height)

        def __eq__(self, other):
            if other == None:
                return False
            return self._key == other._key   # compare items based on their keys

        def __ne__(self, other):
            return not (self == other)       # opposite of __eq__

        def __lt__(self, other):
            return self._key < other._key    # compare items based on their keys

        def __repr__(self):
            return str(self._value)

    def __init__(self):
        """Create an empty map."""
        self._head = self._Node(-math.inf, 'head', 1)   # Head: the first node in a skip list
        # Tail: the last node in a skip list
        self._tail = self._Node(math.inf, 'tail', 1)
        # Initially, there's no item -> head is directly linked to the tail
        self._head._next[0] = self._tail
        self._n = 0                              # Initially, there's no item, so _n = 0
        self._height = 1                         # Initially, the height of a skip list is 1

    def __getitem__(self, k, update=None):
        """Return value associated with key k (raise KeyError if not found)."""
        node, _ = self.do_find(k)
        if node is None:
            raise KeyError(f'{k} not found')
        return node

    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        node, update = self.do_find(k)
        if node:
            node._value = v
            return

        new_height = self.get_random_height()
        height = self._height

        update.extend([self._head for i in range(height, new_height)])
        self._head._next.extend([self._tail for i in range(height, new_height)])
        self._tail._next.extend([None for i in range(height, new_height)])

        self._height = max(self._height, new_height)

        new_node = self._Node(k, v, new_height)
        new_node._next = [update[level]._next[level]
                          for level in range(new_height)]
        for level in range(new_height):
            update[level]._next[level] = new_node

        self._n += 1

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        node, update = self.do_find(k)
        if not node:
            raise KeyError(f'{k} not found')
        for i in reversed(range(len(node._next))):
            update[i]._next[i] = node._next[i]
            if len(self._head._next) > 1 and self._head._next[i] == self._tail:
                self._height -= 1
                head = self._head._next.pop()
                del head
        self._n -= 1
        del node

    def __len__(self):
        """Return number of items in the map."""
        return self._n

    def __iter__(self):
        """Generate iteration of the map's keys."""
        # iterate over the base height (=> height = 0)
        node = self._head._next[0]
        while not node is self._tail:
            yield node._key
            node = node._next[0]

    def get_random_height(self):
        height = 1
        while random.choice([True, False]):
            height += 1
        return height

    def do_find(self, k: int):
        height = self._height
        update = [self._head] * height
        current = self._head
        result = None
        for level in reversed(range(height)):
            node = current
            while node._next[level] != self._tail and node._next[level]._key < k:
                node = node._next[level]

            if node._next[level]._key == k:
                result = node._next[level]
            update[level] = node
        return result, update

    def print_tree(self):
        print('^^^^^^^^^^^^^^^^^^^^^^^^^^')
        node = self._head
        while node != None:
            print('#', end='\t')
            for i in range(self._height):
                lnk = node._next[i] if i < len(node._next) else None

                if node is self._tail:
                    print_val = '+'
                elif lnk == None:
                    print_val = '.'
                elif node._key == -math.inf:
                    print_val = '-'
                elif node._key == math.inf:
                    print_val = '+'
                else:
                    print_val = node._key

                print(print_val, end='\t')
            print()
            node = node._next[0]

        for h in reversed(range(self._height)):
            print(f"At height #{h}, ", end='')
            node = self._head
            while node != None:
                print(node._key, end=' -> ')
                # print(f'h: {h}, node: {node._next}')
                node = node._next[h]
            print()
        print('vvvvvvvvvvvvvvvvvvvvvvvvvv')


# if __name__ == '__main__':
#     sl = SkipList()
#     for i in range(10):
#         sl[i] = chr(65 + i)
#         print(f'sl[{i}] = {sl[i]}')
#     sl.print_tree()
#     for i in range(0, 10, 3):
#         print(i)
#         del sl[i]
#     sl.print_tree()
#     for i in range(10):
#         try:
#             print(sl[i])
#         except KeyError as e:
#             print(e)


In [10]:
from skiplist import SkipList

L = SkipList()
last_key = 0
for i in range(30):
    key = random.randrange(20)
    value = random.randrange(10000)
    print(f'Adding L[{key}] = {value}')
    L[key] = value
    last_key = key
L.print_tree()

print(L[last_key], "# This should be the same to the lastly inserted value")
L[last_key] = 123456789
print(L[last_key], "# This should be 123456789")

print('\nLet\'s iterate through the keys')
for key in L.keys():
    print(key)

print('\nLet\'s delete all the keys, one by one')

del_order = random.sample(list(L), int(len(L)/2))
for k in del_order:
    print(f"Deleting the key {k}")
    del L[k]

L.print_tree()

del_order = random.sample(list(L), len(L))
for k in del_order:
    print(f"Deleting the key {k}")
    del L[k]
L.print_tree()

Adding L[0] = 9888
Adding L[0] = 6543
Adding L[16] = 6633
Adding L[10] = 1300
Adding L[2] = 1952
Adding L[6] = 423
Adding L[8] = 4748
Adding L[8] = 1178
Adding L[9] = 394
Adding L[10] = 1176
Adding L[9] = 1916
Adding L[4] = 4397
Adding L[8] = 8424
Adding L[5] = 2430
Adding L[15] = 6992
Adding L[0] = 8334
Adding L[4] = 8895
Adding L[10] = 9485
Adding L[2] = 3756
Adding L[11] = 4969
Adding L[18] = 726
Adding L[10] = 5203
Adding L[0] = 307
Adding L[13] = 6969
Adding L[7] = 5211
Adding L[5] = 8907
Adding L[16] = 3969
Adding L[9] = 2039
Adding L[9] = 9722
Adding L[12] = 9256
^^^^^^^^^^^^^^^^^^^^^^^^^^
#	-	-	-	-	
#	0	0	.	.	
#	2	2	2	2	
#	4	4	.	.	
#	5	.	.	.	
#	6	6	.	.	
#	7	.	.	.	
#	8	8	.	.	
#	9	.	.	.	
#	10	.	.	.	
#	11	11	.	.	
#	12	.	.	.	
#	13	.	.	.	
#	15	.	.	.	
#	16	16	.	.	
#	18	.	.	.	
#	+	+	+	+	
At height #3, -inf -> 2 -> inf -> 
At height #2, -inf -> 2 -> inf -> 
At height #1, -inf -> 0 -> 2 -> 4 -> 6 -> 8 -> 11 -> 16 -> inf -> 
At height #0, -inf -> 0 -> 2 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 1